<a href="https://colab.research.google.com/github/jvataidee/PesquisaOperacional/blob/master/Aula_PO_Problema_Mochila.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<p align="center"><img src="https://static.wixstatic.com/media/bdd7cb_895c8475a6c94dea9f19f4fde8f978a8~mv2.png" height="70px"><img src="https://static.wixstatic.com/media/bdd7cb_ce7cb603d09d4263921dda3fb68a395f~mv2.png" height="70px"></p>

**by: [João Ataíde](https://www.joaoataide.com)**
#**Knapsack problem com Pyomo**
---


*Knapsack Problem* é um dos problemas mais classicos da pesquisa operacional, sendo esse um das primeiras aplicações do tipos de algoritimos de alocação de recusos, onde trata de maneira simplifica e analoga a idealização de uma mochila de acampamento e quais recursos mais importantes a serem levados, restringido esses a a capacidade de suporte da mochila e o pesos de cada objeto, como pode ser visto na imagem abaixo:

<p align="center"><img src="https://images.tcdn.com.br/img/img_prod/486799/kit_sobrevivencia_de_emergencia_11_em_1_2761_1_20200816231300.jpg" height="250px"></p>

Da mesma maneira, também podem utilizar essa estrutura de problema para resolver problemas de **otimização de portifólio de investimentos** de forma simplificada é claro, pois problemas de otimização de poprtivólio possuem 100x mais restrições. 

Matematicamente falando esse problema da mochila, possui conceito relativamente simple, onde:

Procura-se maximizar o lucro, escolhendo quais dos itens que serão alocados.

$$ maximize \sum\limits_{i=1}^m l_i.x_i$$ 


Restrigindo então que a soma de cada item selecionado seja menor que o custo máximo do investidor

$$Sujeito\ a.\; \sum\limits_{i=1}^m p_i.x_i\leq C $$

$$ x_j \in \{0, 1\},\; j = 1, ... m $$

$ Onde:\\
c - Custo\ de\ cada\ item;\\
C - Custo\ máxima\ do\ investidor\\
l - Lucro\ de\ cada\ item;\\
x - Variácel\ binária ;\\
i - itens.\\
m - total\ de\ itens\\
$

<p align="center"><img src="http://static1.squarespace.com/static/5492d7f4e4b00040889988bd/t/54a3118ae4b0fe11940f9f58/1621524433093/?format=1500w" height="70px"></p>


Para resolver esse problema iremos utilizar a biblioteca Python Optimization
Modeling Objects [Pyomo](http://www.pyomo.org/). Essa biblioteca é especializada em modelagem de problemas de otimização, tal fornece uma sintaxe natural para descrição do modelo, formulação e declarações simplificados. Além disso possui nonceção condiversos solvers do mercado, como GUROBI, CPLEX, GLPK, CBC e outros.

Os dados aqui usados para fazer esse mopdelo foram criados randomicamente seguinte a ordem de grandeza de valores reais, e está  [GitHub](https://github.com/jvataidee/PesquisaOperacional/blob/master/dados.xlsx).







In [1]:
pip install pyomo -q

     |████████████████████████████████| 8.9MB 7.5MB/s 
     |████████████████████████████████| 51kB 9.2MB/s 


In [ ]:
!apt-get install -y -qq glpk-utils -q
!apt-get install -y -qq coinor-cbc -q

In [5]:
#Instalar bibliotes necessárias
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import pyomo.environ as pyo

In [6]:
#Ler dados~
link = 'https://github.com/jvataidee/PesquisaOperacional/blob/master/dados.xlsx?raw=true'
df = pd.read_excel(link)
print("Número de colunas", df.shape[1])
print("Dimensão do problema", df.shape[0])
df.head()

Número de colunas 4
Dimensão do problema 470


,ID,Objeto,Preco,Lucro
0,1,Produto_1,38.902675,1.292848
1,2,Produto_2,60.973190,1.012035
2,3,Produto_3,0.878646,6.785820
3,4,Produto_4,63.398781,1.125438
4,5,Produto_5,122.901828,9.836898


In [7]:
print("Preço total:", df.Preco.sum())

Preço total: 31831.557913880137


In [8]:
#Informação sobre os dados
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 470 entries, 0 to 469
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ID      470 non-null    int64  
 1   Objeto  470 non-null    object 
 2   Preco   470 non-null    float64
 3   Lucro   470 non-null    float64
dtypes: float64(2), int64(1), object(1)
memory usage: 14.8+ KB


In [9]:
#Sumarização estatistica dos dados
df.describe()

,ID,Preco,Lucro
count,470.000000,470.000000,470.000000
mean,235.500000,67.726719,5.868830
std,135.821574,40.354485,3.331029
min,1.000000,0.214309,0.032255
25%,118.250000,30.697567,3.020382
50%,235.500000,68.480253,6.010281
75%,352.750000,102.797657,8.483789
max,470.000000,134.968327,11.990425


In [11]:
#Variáveis de Decisão
C = 5000
m = df.shape[0]  #Dimensão do Problema
custo = list(df.Preco) #Transformar valores em lista
lucro = list(df.Lucro)   #Transformar pesos em lista

In [12]:
# 1 Declaração do modelo
model = pyo.ConcreteModel()

# 2 Declarar as variáveis de decisão e parâmetros
model.I = pyo.RangeSet(m)
model.x = pyo.Var(model.I, within = pyo.Binary)
model.c = pyo.Param(model.I, initialize = lambda model, i: custo[i -1])
model.l = pyo.Param(model.I, initialize = lambda model, i: lucro[i -1])

# 3 Declarar a função objetivo
model.obj = pyo.Objective(expr = sum(model.l[i]*model.x[i] for i in model.I), sense = pyo.maximize)

# 4 Declarar as restrições
model.c1 = pyo.Constraint(expr = sum(model.c[i]*model.x[i] for i in model.I) <= C)

In [13]:
print(model.pprint())

1 RangeSet Declarations
    I : Dimen=1, Size=470, Bounds=(1, 470)
        Key  : Finite : Members
        None :   True : [1:470]

2 Param Declarations
    c : Size=470, Index=I, Domain=Any, Default=None, Mutable=False
        Key : Value
          1 :  38.902674766415856
          2 :   60.97318971056387
          3 :   0.878645958005152
          4 :    63.3987812076952
          5 :  122.90182819611495
          6 :   79.23250657098846
          7 :   91.75568434809855
          8 :   59.89503331942876
          9 :   33.45248512747807
         10 :   22.52079997274259
         11 :   52.14339578258557
         12 :    64.3842991452331
         13 :  103.48418937592879
         14 :  56.692061862743756
         15 :   125.0637127989119
         16 :  107.69596634054925
         17 :  118.40687044604601
         18 :   5.548693025593537
         19 :    83.5065706270292
         20 :  32.109773925786676
         21 :   78.90438762622514
         22 :   71.50094283250722
         23 

In [14]:
#Declarar o Solver
solver_glpk = pyo.SolverFactory('glpk', executable='/usr/bin/glpsol')
solver_cbc =  pyo.SolverFactory('cbc', executable='/usr/bin/cbc')

In [15]:
#Resolver o problema com glpk
result = solver_glpk.solve(model)

In [16]:
model.pprint()

1 RangeSet Declarations
    I : Dimen=1, Size=470, Bounds=(1, 470)
        Key  : Finite : Members
        None :   True : [1:470]

2 Param Declarations
    c : Size=470, Index=I, Domain=Any, Default=None, Mutable=False
        Key : Value
          1 :  38.902674766415856
          2 :   60.97318971056387
          3 :   0.878645958005152
          4 :    63.3987812076952
          5 :  122.90182819611495
          6 :   79.23250657098846
          7 :   91.75568434809855
          8 :   59.89503331942876
          9 :   33.45248512747807
         10 :   22.52079997274259
         11 :   52.14339578258557
         12 :    64.3842991452331
         13 :  103.48418937592879
         14 :  56.692061862743756
         15 :   125.0637127989119
         16 :  107.69596634054925
         17 :  118.40687044604601
         18 :   5.548693025593537
         19 :    83.5065706270292
         20 :  32.109773925786676
         21 :   78.90438762622514
         22 :   71.50094283250722
         23 

In [17]:
# Resultado do modelo
print(result)
print(model.obj())


Problem: 
- Name: unknown
  Lower bound: 1251.23921992776
  Upper bound: 1251.23921992776
  Number of objectives: 1
  Number of constraints: 2
  Number of variables: 471
  Number of nonzeros: 471
  Sense: maximize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 157
      Number of created subproblems: 157
  Error rc: 0
  Time: 0.027416706085205078
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

1251.2392199277615


In [18]:
lista = list(model.x.keys())
for i in lista:
  print(i, "--", model.x[i]())

1 -- 0.0
2 -- 0.0
3 -- 1.0
4 -- 0.0
5 -- 0.0
6 -- 1.0
7 -- 0.0
8 -- 0.0
9 -- 0.0
10 -- 1.0
11 -- 1.0
12 -- 0.0
13 -- 0.0
14 -- 0.0
15 -- 0.0
16 -- 0.0
17 -- 0.0
18 -- 1.0
19 -- 0.0
20 -- 0.0
21 -- 1.0
22 -- 0.0
23 -- 0.0
24 -- 1.0
25 -- 1.0
26 -- 1.0
27 -- 1.0
28 -- 1.0
29 -- 0.0
30 -- 0.0
31 -- 0.0
32 -- 1.0
33 -- 0.0
34 -- 0.0
35 -- 1.0
36 -- 0.0
37 -- 1.0
38 -- 0.0
39 -- 1.0
40 -- 0.0
41 -- 0.0
42 -- 1.0
43 -- 0.0
44 -- 0.0
45 -- 0.0
46 -- 0.0
47 -- 0.0
48 -- 1.0
49 -- 0.0
50 -- 0.0
51 -- 0.0
52 -- 1.0
53 -- 0.0
54 -- 0.0
55 -- 1.0
56 -- 0.0
57 -- 0.0
58 -- 1.0
59 -- 0.0
60 -- 0.0
61 -- 0.0
62 -- 0.0
63 -- 0.0
64 -- 1.0
65 -- 0.0
66 -- 0.0
67 -- 1.0
68 -- 1.0
69 -- 1.0
70 -- 0.0
71 -- 0.0
72 -- 0.0
73 -- 1.0
74 -- 1.0
75 -- 1.0
76 -- 1.0
77 -- 0.0
78 -- 1.0
79 -- 0.0
80 -- 1.0
81 -- 1.0
82 -- 1.0
83 -- 1.0
84 -- 1.0
85 -- 1.0
86 -- 0.0
87 -- 1.0
88 -- 0.0
89 -- 0.0
90 -- 0.0
91 -- 0.0
92 -- 0.0
93 -- 0.0
94 -- 0.0
95 -- 1.0
96 -- 0.0
97 -- 1.0
98 -- 1.0
99 -- 0.0
100 -- 0.0
101 -- 0